## 20221209

In [86]:
# import

from collections import defaultdict, namedtuple
from htbuilder import div, big, h2, styles
from htbuilder.units import rem
from math import floor
import altair as alt
import datetime
import functools
import numpy as np
import pandas as pd
import re
import streamlit as st
import time
import plotly.express as px
from vega_datasets import data
from st_aggrid import AgGrid

from streamlit_app import instruction as instr
from streamlit_app import form as form
from streamlit_app import custom_display as cd

import altair_viewer
alt.renderers.enable('altair_viewer', inline=True)
# alt.renderers.enable('notebook')
# alt.renderers.enable('mimetype')

RendererRegistry.enable('altair_viewer')

In [18]:
COLOR_RED = "#ED2E24"
COLOR_DARK_BLUE = "#052A5F"
COLOR_BLUE = "#0476BF"

### Baseline MA

In [43]:
# MA 5-10 chart

path = 'models-v2-lle/testset_w_lle.csv'
df = pd.read_csv(path)
df.head()

,attendance,is_federal_holiday,venue,on_grass,temperature,wind_speed,team1_name,team1_pre_win,team1_pre_loss,team1_pre_win_pct,...,start_hour,start_hour_label_afternoon,start_hour_label_evening,start_hour_label_night,start_hour_label_noon,game_page_url,start_time,previous_5_to_10MA,lle1,lle2
0,35112,0,Wrigley Field,1,-2.786672,1.292848,MIL,-1.62645,-1.654514,0.0,...,13,0,0,0,1,https://www.baseball-reference.com/boxes/CHN/C...,2022-04-07 13:20:00,38208.209877,0.009912,-0.006604
1,28459,0,Kauffman Stadium,1,-2.503546,1.688830,CLE,-1.62645,-1.654514,0.0,...,15,1,0,0,0,https://www.baseball-reference.com/boxes/KCA/K...,2022-04-07 15:10:00,18495.737500,0.007218,0.002121
2,46256,0,Busch Stadium III,1,-2.220421,2.480794,PIT,-1.62645,-1.654514,0.0,...,15,1,0,0,0,https://www.baseball-reference.com/boxes/SLN/S...,2022-04-07 15:15:00,42967.814815,0.007821,0.004266
3,44723,0,Angel Stadium of Anaheim,1,1.837706,-0.291081,HOU,-1.62645,-1.654514,0.0,...,18,0,1,0,0,https://www.baseball-reference.com/boxes/ANA/A...,2022-04-07 18:38:00,37321.135802,0.008651,-0.001540
4,35508,0,Chase Field,0,1.837706,-0.291081,SDP,-1.62645,-1.654514,0.0,...,18,0,1,0,0,https://www.baseball-reference.com/boxes/ARI/A...,2022-04-07 18:40:00,26364.320988,0.008440,-0.000359


In [85]:
home_team_name = 'Arizona Diamondbacks (ARI)'
# extract
home_team_name = home_team_name.split(' (')[1].replace(')', '')
home_team_name

'ARI'

In [44]:
home_team_name = "ARI"

df_home = df[df["team2_name"] == home_team_name]
df_home.head(3)

,attendance,is_federal_holiday,venue,on_grass,temperature,wind_speed,team1_name,team1_pre_win,team1_pre_loss,team1_pre_win_pct,...,start_hour,start_hour_label_afternoon,start_hour_label_evening,start_hour_label_night,start_hour_label_noon,game_page_url,start_time,previous_5_to_10MA,lle1,lle2
4,35508,0,Chase Field,0,1.837706,-0.291081,SDP,-1.626450,-1.654514,0.0,...,18,0,1,0,0,https://www.baseball-reference.com/boxes/ARI/A...,2022-04-07 18:40:00,26364.320988,0.008440,-0.000359
15,17297,0,Chase Field,0,2.026456,-1.083045,SDP,-1.626450,-1.613733,0.0,...,18,0,1,0,0,https://www.baseball-reference.com/boxes/ARI/A...,2022-04-08 18:40:00,26364.320988,0.008948,-0.001030
29,19504,0,Chase Field,0,2.215206,-1.281036,SDP,-1.586842,-1.613733,0.5,...,17,0,1,0,0,https://www.baseball-reference.com/boxes/ARI/A...,2022-04-09 17:10:00,26364.320988,0.008659,-0.000620


In [102]:
# baseline value 

# given date and team name 

def nearest(items, pivot):
    target_date = pd.to_datetime(min([i for i in items if i <= pivot], key=lambda x: abs(x - pivot)))
    return target_date


date_str = np.datetime64('2022-04-20')
team_name = "ARI"

# get baseline value
baseline_value = df_home[(df_home["date"] == nearest(df_home['date'].values, date_str)) & (df_home["team2_name"] == team_name)]["previous_5_to_10MA"].values[0]
baseline_value

26364.32098765432

In [48]:
df_home["date"] = pd.to_datetime(df_home["start_time"].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S").date()))
df_home["date"]

/var/folders/j3/sr9rbcw10dzch_cpwn25txkh0000gn/T/ipykernel_82368/3518013312.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_home["date"] = pd.to_datetime(df_home["start_time"].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S").date()))


4      2022-04-07
15     2022-04-08
29     2022-04-09
41     2022-04-10
67     2022-04-12
          ...    
2147   2022-09-17
2161   2022-09-18
2229   2022-09-23
2244   2022-09-24
2256   2022-09-25
Name: date, Length: 81, dtype: datetime64[ns]

In [65]:
source = df_home

bar = alt.Chart(source).mark_bar(color=COLOR_BLUE).encode(  # type: ignore
    x = alt.X('date:O', axis=alt.Axis(title='Date', labelAngle=30)), # type: ignore
    y = alt.Y('attendance:Q', axis=alt.Axis(title='Attendance')), # type: ignore
    tooltip=[
        alt.Tooltip('attendance', title='Attendance'), # type: ignore
        alt.Tooltip('previous_5_to_10MA', title='Previous 5-10 MA'), # type: ignore
        alt.Tooltip('date', title='Date'), # type: ignore
    ]
)

line = alt.Chart(source).mark_line(color=COLOR_RED).encode( # type: ignore
    x = 'date:T',
    y = 'previous_5_to_10MA:Q',
)


tmp_plt = (bar + line).interactive()
altair_viewer.display(tmp_plt, inline=True)

In [24]:
source = data.wheat()

bar = alt.Chart(source).mark_bar(color=COLOR_BLUE).encode(  # type: ignore
    x='year:O',
    y='wheat:Q',
    tooltip=['year', 'wheat']
)

line = alt.Chart(source).mark_line(color=COLOR_RED).transform_window(  # type: ignore
    # The field to average
    rolling_mean='mean(wheat)',
    # The number of values before and after the current value to include.
    frame=[-9, 0]
).encode(
    x='year:O',
    y='rolling_mean:Q',
    tooltip=['year']
)

tmp_plt = (bar + line).interactive()
altair_viewer.display(tmp_plt, inline=True)

/Users/brian/.local/share/virtualenvs/SLDL-MLB-ATT-Prediction-gFjMjB_x/lib/python3.9/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


2022-12-09 11:15:14.204 INFO    tornado.access: 200 GET /scripts/vega-embed.js (::1) 0.57ms
INFO:tornado.access:200 GET /scripts/vega-embed.js (::1) 0.57ms
2022-12-09 11:15:14.270 INFO    tornado.access: 200 GET /scripts/vega.js (::1) 1.75ms
INFO:tornado.access:200 GET /scripts/vega.js (::1) 1.75ms
2022-12-09 11:15:14.275 INFO    tornado.access: 200 GET /scripts/vega-lite.js (::1) 3.16ms
INFO:tornado.access:200 GET /scripts/vega-lite.js (::1) 3.16ms


### Feature Importance

In [68]:
path = 'streamlit_app/instruction/model_feature_importance.xlsx'
summary_df = pd.read_excel(path, sheet_name='Summary', engine='openpyxl')
raw_df = pd.read_excel(path, sheet_name='Ft. Importance Table', engine='openpyxl')

In [69]:
raw_df.head()

,Model,ID,Feature,Processed Feature Name,Metric
0,XGBoost,1,team2_name_TBR,Home: TBR,0.111499
1,XGBoost,2,team2_name_LAD,Home: LAD,0.097885
2,XGBoost,3,team2_name_STL,Home: STL,0.090991
3,XGBoost,4,season_type_regular,Season: Regular,0.072797
4,XGBoost,5,home_team_avg_att_last_year,Metric: Home Team Avg. Att. Last Y.,0.047648


In [71]:
model_df = raw_df[raw_df["Model"] == "XGBoost"]

In [80]:
source = model_df

tmp_plt = alt.Chart(source).mark_bar(color=COLOR_BLUE).encode(  # type: ignore
    x='Metric:Q',
    y=alt.Y('Processed Feature Name:N', sort='-x', axis=alt.Axis(labelLimit=100)), # type: ignore
    tooltip=[
        'Model',
        'Metric', 
             alt.Tooltip('Processed Feature Name', title='Feature Name'), # type: ignore
             alt.Tooltip('Feature', title='Raw Feature Name'), # type: ignore
             
            ]
)

# st.altair_chart(tmp_plt, use_container_width=True)
altair_viewer.display(tmp_plt, inline=True)

/Users/brian/.local/share/virtualenvs/SLDL-MLB-ATT-Prediction-gFjMjB_x/lib/python3.9/site-packages/altair/utils/core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


## Before 20221209

In [22]:
import pandas as pd
data = pd.read_pickle('testset.pickle')

In [23]:
# print first 5 data keys
print(len(list(data.keys())))

2470


In [25]:
list(list(data.keys())[2450:2455])

['https://www.baseball-reference.com/boxes/PHI/PHI202210150.shtml',
 'https://www.baseball-reference.com/boxes/SDN/SDN202210150.shtml',
 'https://www.baseball-reference.com/boxes/SEA/SEA202210150.shtml',
 'https://www.baseball-reference.com/boxes/CLE/CLE202210160.shtml',
 'https://www.baseball-reference.com/boxes/NYA/NYA202210180.shtml']

In [26]:
data['https://www.baseball-reference.com/boxes/PHI/PHI202210150.shtml']

{'game_page_url': 'https://www.baseball-reference.com/boxes/PHI/PHI202210150.shtml',
 'have_att': True,
 'start_time': datetime.datetime(2022, 10, 15, 14, 7),
 'day': 'Saturday',
 'month': 'October',
 'is_federal_holiday': 0,
 'duration': datetime.timedelta(seconds=11880),
 'venue': 'Citizens Bank Park',
 'at_night': False,
 'on_grass': True,
 'weather_description': ' 72° F, Wind 12mph out to Centerfield, Sunny, No Precipitation.',
 'temperature': 72,
 'wind_speed': 12,
 'weather': ' Sunny',
 'precipitation': 1,
 'umpires': ' HP - Stu Scheurwater, 1B - Chad Fairchild, 2B - Ryan Blakney, 3B - David Rackley, LF - Nic Lentz, RF - Bill Miller.',
 'attendance': 45660,
 'team1_name': 'AtlantaBraves',
 'team1_pre_win': 101,
 'team1_pre_loss': 60,
 'team1_pre_win_pct': 0.6273291925465838,
 'team1_players':                  Batting AB  R  H RBI BB SO PA    BA   OBP  ...     WPA  \
 0    Ronald Acuna Jr. RF  4  0  0   0  0  3  4  .333  .412  ...  -0.067   
 1      Dansby Swanson SS  4  0  0   0 